In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("BatterySimV3.py"))
parent = os.path.dirname(os.path.dirname(current))
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel
from Logic import logic_bat, logic_DP, logic_actions
from Funcs_Logic_DP import (get_price, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)

In [2]:
house = "h16"
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001
...,...,...,...,...
2023-01-09 16:00:00,0.1,7.6,-7.5,1204.510010
2023-01-09 17:00:00,0.0,10.0,-10.0,1296.579956
2023-01-09 18:00:00,0.0,9.1,-9.1,1354.890015
2023-01-09 19:00:00,0.0,11.1,-11.1,1266.089966


## Running the logic models and DP

In [3]:
#"Input" variables
avg_yearly_cost_dict = {}
N = 24*7
bat_capacity=13
Start="2022-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N, freq="h")[-1]

In [4]:
# Define battery with capcacity 0, same as no battery
battery_no_bat = Battery(max_capacity=0,max_charge=0)

series_no_battery = logic_rollout(merged.loc[Start:End].copy(), battery_no_bat, logic_bat)

charge_list_no_bat = battery_no_bat.charge_list

avg_yearly_cost_dict["No battery"] = (print_price_summary(series_no_battery), 
                                      np.sum(charge_list_no_bat[charge_list_no_bat>0]), 
                                      np.sum(charge_list_no_bat[charge_list_no_bat<=0]))

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  802.0  DKK
Average cost per year is: 42077.0 DKK
Number of kwh purchased in the period: 456.29999999990105
Average number of kwh purchased per year: 23951.651497000792
Average number of kwh sold per year: 34334.37485029512


In [5]:
logic_series_print(series_no_battery)

hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:   0.7333, -3.1000, -3.1000,  0.0000,  0.0000,  0.0000,  0.0000,  2.2731, 36.5057
    8:   0.6735, -3.3000, -3.3000,  0.0000,  0.0000,  0.0000,  0.0000,  2.2224, 38.7281
    9:   0.7013, -3.2000, -3.2000,  0.0000,  0.0000,  0.0000,  0.0000,  2.2443, 40.9724
   10:   0.7004, -5.4000, -5.4

  125:   1.4062, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  4.9218,617.5565
  126:   2.0425, -2.7000, -2.7000,  0.0000,  0.0000,  0.0000,  0.0000,  5.5147,623.0712
  127:   2.4933, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000, 10.4718,633.5430
  128:   2.5719, -3.1000, -3.1000,  0.0000,  0.0000,  0.0000,  0.0000,  7.9729,641.5159
  129:   2.4547,  3.7000,  3.7000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9082,640.6076
  130:   2.2074,  7.6000,  7.6000,  0.0000,  0.0000,  0.0000,  0.0000, -1.6776,638.9300
  131:   1.7718, 12.0000, 12.0000,  0.0000,  0.0000,  0.0000,  0.0000, -2.1261,636.8039
  132:   1.1282,  9.9000,  9.9000,  0.0000,  0.0000,  0.0000,  0.0000, -1.1170,635.6869
  133:   1.5413, 20.0000, 20.0000,  0.0000,  0.0000,  0.0000,  0.0000, -3.0825,632.6044
  134:   1.4980, 18.1000, 18.1000,  0.0000,  0.0000,  0.0000,  0.0000, -2.7115,629.8929
  135:   1.4753, 20.5000, 20.5000,  0.0000,  0.0000,  0.0000,  0.0000, -3.0245,626.8685
  136:   1.3316, 18.9000, 18.900

In [6]:
# Define battery with capcacity
battery = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery = logic_rollout(merged.loc[Start:End].copy(), battery, logic_bat)

charge_list_bat = battery.charge_list

avg_yearly_cost_dict["Simple logic"] = (print_price_summary(series_battery), 
                                        np.sum(charge_list_bat[charge_list_bat>0]), 
                                        np.sum(charge_list_bat[charge_list_bat<=0]))

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  652.0  DKK
Average cost per year is: 34228.0 DKK
Number of kwh purchased in the period: 382.5999999999011
Average number of kwh purchased per year: 20083.0634730487
Average number of kwh sold per year: 28339.900598798115


In [7]:
logic_series_print(series_battery)

hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:   0.7333, -3.1000, -3.1000,  0.0000,  0.0000,  0.0000,  0.0000,  2.2731, 36.5057
    8:   0.6735, -3.3000, -3.3000,  0.0000,  0.0000,  0.0000,  0.0000,  2.2224, 38.7281
    9:   0.7013, -3.2000, -3.2000,  0.0000,  0.0000,  0.0000,  0.0000,  2.2443, 40.9724
   10:   0.7004, -5.4000, -5.4

In [8]:
DPbat = Battery(max_capacity=bat_capacity)

DP = DPModel(N, Start, End, house, merged, deepcopy(DPbat))

J1, pi = DP_stochastic(DP)

0   

In [9]:
J, traj, actions = policy_rollout(DP,pi=lambda x, k: pi[k][x],x0=0.0)
J

596.1695734675003

In [10]:
series_DP_battery = logic_rollout(merged.loc[Start:End].copy(), DPbat, logic_DP, actions)

charge_list_DP_bat = DPbat.charge_list

avg_yearly_cost_dict["DP"] = (print_price_summary(series_DP_battery), 
                                      np.sum(charge_list_DP_bat[charge_list_DP_bat>0]), 
                                      np.sum(charge_list_DP_bat[charge_list_DP_bat<=0]))

Cost for period: 2022-06-19 00:00:00 to 2022-06-25 23:00:00 is:  605.0  DKK
Average cost per year is: 31733.0 DKK
Number of kwh purchased in the period: 393.6999999999302
Average number of kwh purchased per year: 20665.713772451425
Average number of kwh sold per year: 29216.500598799637


In [11]:
logic_series_print(series_DP_battery)

hour     price    yield    surplus  buy      charge   before   after    cost     cumsum  
    0:   1.5404, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  8.4721,  8.4721
    1:   1.4134, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  7.4911, 15.9632
    2:   1.2313, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  5.1715, 21.1347
    3:   1.0400, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  3.9519, 25.0866
    4:   0.8971, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  3.4091, 28.4957
    5:   0.8254, -3.4000, -3.4000,  0.0000, -0.0000,  0.0000,  0.0000,  2.8065, 31.3022
    6:   0.8373, -3.5000, -3.5000,  0.0000, -0.0000,  0.0000,  0.0000,  2.9304, 34.2326
    7:   0.7333, -3.1000, -3.1000,  0.0000, -0.0000,  0.0000,  0.0000,  2.2731, 36.5057
    8:   0.6735, -3.3000, -3.3000,  0.0000, -0.0000,  0.0000,  0.0000,  2.2224, 38.7281
    9:   0.7013, -3.2000, -3.2000,  0.0000, -0.0000,  0.0000,  0.0000,  2.2443, 40.9724
   10:   0.7004, -5.4000, -5.4